In [5]:
# Código completo para carregar o dataset, filtrar, treinar o modelo de similaridade e recomendar com base em 3 títulos

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Carregar dataset
df = pd.read_csv("meu_dataset_completo.csv")

# Filtrar apenas filmes com sinopse e imagem
df_filtrado = df.dropna(subset=["plot", "poster"]).reset_index(drop=True)

# Criar matriz TF-IDF com base na sinopse
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_filtrado['plot'])

# Função de recomendação
def recomendar_filmes(titulos_usuario, top_n=15):
    indices_input = []
    for titulo in titulos_usuario:
        match = df_filtrado[df_filtrado['title'].str.lower() == titulo.lower()]
        if match.empty:
            raise ValueError(f"Título '{titulo}' não encontrado no dataset com imagem.")
        indices_input.append(match.index[0])

    vetor_soma = np.sum(tfidf_matrix[indices_input], axis=0)
    vetor_soma = np.asarray(vetor_soma)  # Convert to numpy array
    similaridades = cosine_similarity(vetor_soma, tfidf_matrix).flatten()
    indices_recomendados = np.argsort(similaridades)[::-1]
    indices_recomendados = [idx for idx in indices_recomendados if idx not in indices_input][:top_n]

    return df_filtrado.loc[indices_recomendados, ['title', 'rating', 'plot', 'poster']]

# Exemplo de uso
if __name__ == "__main__":
    titulos = ["Hamlet", "Miss Jerry", "The Prodigal Son"]
    recomendacoes = recomendar_filmes(titulos)
    for i, row in recomendacoes.iterrows():
        print(f"🎬 {row['title']}\n⭐ Rating: {row['rating']}\n📝 {row['plot']}\n🖼️ Poster: {row['poster']}\n")



🎬 Hamlet, Prince of Denmark
⭐ Rating: 4.4
📝 Hamlet suspects his uncle has murdered his father to claim the throne of Denmark and the hand of Hamlet's mother, but the prince cannot decide whether or not he should take vengeance.
🖼️ Poster: https://m.media-amazon.com/images/M/MV5BZmQ0ZmRlOGUtOGJmNi00NDhmLWJiZTYtNTJmNzhkOWY3YzE1XkEyXkFqcGdeQXVyOTk0NTYwMzA@._V1_SX300.jpg

🎬 The Corbett-Fitzsimmons Fight
⭐ Rating: 5.2
📝 Documentary film depicting the 1897 boxing match between James J. Corbett and Bob Fitzsimmons in Carson City, Nevada on St. Patrick's Day. Originally running for more than 100 minutes, it is the world's first feature film.
🖼️ Poster: https://m.media-amazon.com/images/M/MV5BMTAwOWQ2MWQtNDc3ZS00ZGI2LTkzZWQtMjhkMzFiOTM2MzMyXkEyXkFqcGc@._V1_SX300.jpg

🎬 Dante's Inferno
⭐ Rating: 7.0
📝 1911 silent film and Italy's first full-length feature film, loosely adapted from "Inferno", the first canticle of Dante Alighieri's "Divine Comedy". It chronicles Dante's travel through the Circle

In [ ]:
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

CORS(app)

# Carregar o dataset
df = pd.read_csv("meu_dataset_completo.csv")
df_filtrado = df.dropna(subset=["plot", "poster"]).reset_index(drop=True)

# Criar matriz TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df_filtrado['plot'])

# Função de recomendação
def recomendar_filmes(titulos_usuario, top_n=9):
    indices_input = []
    for titulo in titulos_usuario:
        match = df_filtrado[df_filtrado['title'].str.lower() == titulo.lower()]
        if match.empty:
            raise ValueError(f"Título '{titulo}' não encontrado no dataset com imagem.")
        indices_input.append(match.index[0])

    vetor_soma = np.asarray(np.sum(tfidf_matrix[indices_input], axis=0))
    similaridades = cosine_similarity(vetor_soma, tfidf_matrix).flatten()
    indices_recomendados = np.argsort(similaridades)[::-1]
    indices_recomendados = [idx for idx in indices_recomendados if idx not in indices_input][:top_n]

    resultados = df_filtrado.loc[indices_recomendados, ['title', 'rating', 'plot', 'poster']]
    return resultados.to_dict(orient='records')

# Rota principal (caso queira renderizar o HTML diretamente)
@app.route("/")
def index():
    return render_template("index.html")

# Rota de recomendação
@app.route("/recomendar", methods=["POST"])
def recomendar():
    dados = request.get_json()
    filmes = dados.get("filmes", [])

    try:
        resultados = recomendar_filmes(filmes)
        return jsonify({"recomendacoes": resultados})
    except Exception as e:
        print("Erro:", str(e))
        return jsonify({"erro": str(e)}), 400

if __name__ == "__main__":
    app.run(debug=True)
